In [1]:
import pandas as pd
data = pd.read_csv('preprocessed_data.csv')
data.head()

,temperature_2m,relative_humidity_2m,dew_point_2m,surface_pressure,cloud_cover,wind_speed_10m,weather_group,humidity_change,hour_sin,hour_cos,pressure_trend_6h,month_sin,month_cos,press_std_12h,hum_max_6h,temp_diff_3h
0,24.7,80,21.0,1013.3,96,10.4,Cloudy,4.0,-0.965926,-2.588190e-01,-2.2,0.5,0.866025,1.368808,80.0,-0.5
1,24.2,82,21.0,1013.6,87,6.8,Cloudy,2.0,-1.000000,-1.836970e-16,-1.3,0.5,0.866025,1.395990,82.0,-1.1
2,23.5,87,21.2,1014.1,40,5.3,Cloudy,5.0,-0.965926,2.588190e-01,0.1,0.5,0.866025,1.393383,87.0,-1.6
3,23.4,87,21.0,1015.3,83,5.6,Drizzle,0.0,-0.866025,5.000000e-01,2.1,0.5,0.866025,1.297083,87.0,-1.3
4,23.0,90,21.2,1015.4,86,5.9,Drizzle,3.0,-0.707107,7.071068e-01,2.7,0.5,0.866025,1.161862,90.0,-1.2


In [2]:
data.columns

Index(['temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m', 'weather_group',
       'humidity_change', 'hour_sin', 'hour_cos', 'pressure_trend_6h',
       'month_sin', 'month_cos', 'press_std_12h', 'hum_max_6h',
       'temp_diff_3h'],
      dtype='object')

In [3]:
X = data.drop(columns=['weather_group'])
y = data['weather_group'].astype('category').cat.codes # Chuyển nhãn chữ sang số

In [4]:
X = data.drop(columns=['weather_group', 'hum_max_6h'])
y = data['weather_group'].astype('category').cat.codes

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle=False)


In [ ]:
from sklearn.preprocessing import RobustScaler

robust_features = [
    'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
    'surface_pressure', 'wind_speed_10m', 'humidity_change', 
    'pressure_trend_6h', 'press_std_12h', 'temp_diff_3h'
]

robust_scaler = RobustScaler()

X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

X_train_scaled[robust_features] = robust_scaler.fit_transform(X_train[robust_features])
X_val_scaled[robust_features] = robust_scaler.transform(X_val[robust_features])
X_test_scaled[robust_features] = robust_scaler.transform(X_test[robust_features])

## 4. Softmax Regression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
from imblearn.pipeline import Pipeline
import time
import numpy as np

### 4.1 Tìm kiếm siêu tham số (Hyperparameter Tuning)

Để tìm ra bộ tham số tối ưu cho mô hình Softmax Regression, chúng tôi sử dụng kỹ thuật **Grid Search** kết hợp với **Time Series Cross Validation**.

**Phương pháp:**
- **Grid Search**: Thử nghiệm exhaustive trên một lưới các giá trị tham số.
- **TimeSeriesSplit**: Vì dữ liệu là chuỗi thời gian, việc chia tập validation cần tôn trọng thứ tự thời gian. `TimeSeriesSplit` chia dữ liệu thành *k* fold liên tiếp, trong đó fold thứ *i* dùng làm train cho fold thứ *i+1* (validation), đảm bảo không dùng tương lai để dự đoán quá khứ.

**Các siêu tham số được tinh chỉnh:**
1.  **C** (Regularization Strength): [0.001, 0.01, 1, 10, 100]
2.  **solver**: ['lbfgs', 'newton-cg']
3.  **max_iter**: 1000


In [8]:
# Chuẩn bị dữ liệu: Gộp Train và Val để GridSearch với TimeSeriesSplit
X_train_val = pd.concat([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

In [ ]:
# Time Series Cross Validation
tscv = TimeSeriesSplit(n_splits=5)

# Thử nghiệm các tổ hợp trọng số
custom_weights = [{0: 1, 1: 1.2, 2: 1.5}, {0: 1, 1: 1.5, 2: 2.0}]

# Tạo pipeline: Poly -> SMOTE -> LogisticRegression
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(max_iter=3000))
])

param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__class_weight': [None, 'balanced'] + custom_weights,
    'classifier__solver': ['lbfgs', 'newton-cg'],
    'poly__interaction_only': [False, True]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=tscv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Thực hiện tìm kiếm
start_search = time.time()
grid_search.fit(X_train_val, y_train_val)
end_search = time.time()

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score: {grid_search.best_score_:.4f}")
print(f"Search time: {end_search - start_search:.2f}s")

best_model_softmax = grid_search.best_estimator_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
# Xem tầm quan trọng của các biến trong Softmax (đã qua Poly)
classifier = best_model_softmax.named_steps['classifier']
poly = best_model_softmax.named_steps['poly']
feature_names = poly.get_feature_names_out(X.columns)

for i, class_label in enumerate(classifier.classes_):
    print(f"\nTop features cho lớp {class_label}:")
    top_indices = np.argsort(np.abs(classifier.coef_[i]))[-5:]
    for idx in top_indices[::-1]:
        print(f"{feature_names[idx]}: {classifier.coef_[i][idx]:.4f}")


Top features cho lớp 0:
relative_humidity_2m: -1.5847
hour_cos: 0.8635

Top features cho lớp 1:
surface_pressure: 0.8766
temperature_2m: 0.7589

Top features cho lớp 2:
relative_humidity_2m: 1.5661
wind_speed_10m: 0.9165


### 4.2 Kiểm thử mô hình (Model Evaluation)

Sau khi chọn được mô hình tốt nhất, chúng tôi tiến hành đánh giá trên tập kiểm thử (Test Set) hoàn toàn độc lập.

**Các chỉ số đánh giá (Metrics):**
1.  **Accuracy (Độ chính xác)**: Tỷ lệ số mẫu dự đoán đúng trên tổng số mẫu. Đo lường hiệu suất tổng quan.
2.  **Training Time & Inference Time**: 
    - *Training Time*: Thời gian huấn luyện lại mô hình tốt nhất.
    - *Inference Time*: Thời gian mô hình dự đoán trên tập test.
3.  **Precision (Độ chính xác của từng lớp)**: Trong số các mẫu được dự đoán là lớp A, bao nhiêu % thực sự là lớp A. Quan trọng khi muốn giảm thiểu báo động giả (False Positives).
4.  **Recall (Độ phủ)**: Trong số các mẫu thực sự là lớp A, bao nhiêu % được mô hình phát hiện đúng. Quan trọng khi muốn tránh bỏ sót (False Negatives).
5.  **F1-Score**: Trung bình điều hòa của Precision và Recall. Cân bằng giữa độ chính xác và độ phủ, hữu ích khi dữ liệu mất cân bằng.
6.  **Confusion Matrix**: Bảng ma trận hiển thị chi tiết sự nhầm lẫn giữa các lớp.

In [ ]:
# Huấn luyện lại mô hình tốt nhất (hoặc sử dụng kết quả từ GridSearch đã fit trên Train+Val)
# Ở đây GridSearch refit=True mặc định sẽ fit lại trên toàn bộ dữ liệu đưa vào (Train + Val)
# Đo thời gian Inference
start_inf = time.time()
y_pred = best_model_softmax.predict(X_test_scaled)
end_inf = time.time()
inference_time = end_inf - start_inf

# Đánh giá trên Train và Test
train_acc = best_model_softmax.score(X_train_val, y_train_val)
test_acc = accuracy_score(y_test, y_pred)

print(f"Training Accuracy (Result of GridSearch refit): {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")
print(f"Inference Time (on test set): {inference_time:.4f}s")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

Training Accuracy (Result of GridSearch refit): 0.7774
Testing Accuracy: 0.6266
Inference Time (on test set): 0.0033s

Classification Report:

              precision    recall  f1-score   support

           0       0.69      0.73      0.71      1673
           1       0.55      0.57      0.56      1366
           2       0.65      0.43      0.52       472

    accuracy                           0.63      3511
   macro avg       0.63      0.58      0.59      3511
weighted avg       0.63      0.63      0.62      3511


Confusion Matrix:

[[1225  438   10]
 [ 494  772  100]
 [  66  203  203]]
